In [1]:
# v1.2
# double checks for entry
# has stop loss built in

In [2]:
# imports and requirements
import json
import requests
import pandas as pd
import numpy as np
import ccxt
import time
import datetime as dt
import matplotlib.pyplot as plt

from binance.client import Client
from binance.enums import *

In [3]:
# init client
apiKey = 'Xo3vFSsM4XoYge93eqCA3gxmHQTN3rWqOs1LtUsHK9UQDKFcvRp1hMRmIiFIolsh'
apiSecret = 'bsY8Jh1A4jFNv8xLNWbRBff8MPoKQKeyB601YFoSRMiozHRZ4hj3m7pJFcMBIjy5'

# python-binance init
client = Client(apiKey, apiSecret)
# ccxt init
exchange = ccxt.binance({
    'apiKey': 'Xo3vFSsM4XoYge93eqCA3gxmHQTN3rWqOs1LtUsHK9UQDKFcvRp1hMRmIiFIolsh',
    'secret': 'bsY8Jh1A4jFNv8xLNWbRBff8MPoKQKeyB601YFoSRMiozHRZ4hj3m7pJFcMBIjy5',
    'enableRateLimit': True,  # as required by the Manual
    'options': { 'defaultType': 'future' }
})

In [ ]:
# inputs
    
leg1 = 'ONTUSDT'
leg2 = 'QTUMUSDT'
timeframe = '30m'
emalen = '20'
legval = float(input("Leg Value: "))
maxWDD = float(input("Max wdd on 50k"))

# convert legs
if leg1 == 'ONTUSDT':
    newLeg1 = 'ONT/USDT'
    prec1 = 4
elif leg1 == 'ETHUSDT':
    newLeg1 = 'ETH/USDT'
    prec1 = 2
elif leg1 == 'BCHUSDT':
    newLeg1 = 'BCH/USDT'
    prec1 = 2
elif leg1 == 'TRXUSDT':
    newleg1 = 'TRX/USDT'
    prec1 = 5
    
if leg2 == 'QTUMUSDT':
    newLeg2 = 'QTUM/USDT'
    prec2 = 3
elif leg2 == 'XTZUSDT':
    newLeg2 = 'XTZ/USDT'
    prec2 = 3
elif leg2 == 'LTCUSDT':
    newLeg2 = 'LTC/USDT'
    prec2 = 2
elif leg2 == 'BATUSDT':
    newLeg2 = 'BAT/USDT'
    prec2 = 4

print("New leg 1:", format(newLeg1))
print("New leg 2:", format(newLeg2))


Leg Value: 5


In [ ]:
# create dataframe
def dataframe(symbol, interval, timeframe):
    rootUrl = 'https://api.binance.com/api/v1/klines'
    url = rootUrl + '?symbol=' + symbol + '&interval=' + interval + '&limit=' + timeframe
    data = json.loads(requests.get(url).text)
    df = pd.DataFrame(data)
    df.columns = ['open_time',
                 'o', 'h', 'l', 'c', 'v',
                 'closeTime', 'qav', 'numTrades',
                 'takerBase_Bol', 'takerQuoteVol', 'ignore']
    df.index = [dt.datetime.fromtimestamp(x/1000.0) for x in df.closeTime]
    return df

dataframe(leg1, timeframe, emalen)

In [ ]:
# hedge price function
def priceChart():
    leg1Chart= dataframe(leg1,timeframe, emalen)
    leg2Chart = dataframe(leg2,timeframe, emalen)

    leg1ChartClose = leg1Chart['c'].astype('float')
    leg2ChartClose = leg2Chart['c'].astype('float')

    hedgeChart = leg1ChartClose / leg2ChartClose
    return hedgeChart

# prints price chart
priceChart().plot(figsize=(16,9))

In [ ]:
# current zscore
def currZscore():
    currStd = priceChart().std()
    currMean = priceChart().mean()
    a = priceChart().tolist()
    x = a[-1:]
    
    y = [float(i) for i in x]

    zscore = (y[0] - currMean)/currStd
    return round(zscore,5)

print("Current Z-Score = %s" % currZscore())

In [ ]:
# short buy and long exit
def shortSide(leg1, leg2, legval, maxWDD):
    
    print("Short side function: ...")
    # amount for each leg
    priceLeg1 = dataframe(leg1,timeframe,emalen)['c'].astype('float')[-1].tolist()
    amountLeg1 = round(legval/priceLeg1,1)
    print ("Amount of leg1: {}". format(amountLeg1))
    
    priceLeg2 = dataframe(leg2,timeframe,emalen)['c'].astype('float')[-1].tolist()
    amountLeg2 = round(legval/priceLeg2,1)
    print ("Amount of leg2: {}". format(amountLeg2))
    print("-----")


    # short leg1    
    order1 = exchange.createOrder(newLeg1, 'market', 'sell', amountLeg1)
    print("Shorted leg 1")
    print(order1)
    # long leg 2
    order2 = exchange.createOrder(newLeg2, 'market', 'buy', amountLeg2)
    print("Longed leg 2")
    print(order2)
    
    inPos = True
    print ("inPos status before exit checks: {}". format(inPos))
    print("-----")

    
    # exit conds
    while (inPos == True):
        exZscore = currZscore()
        currPrice1 = dataframe(leg1,timeframe,emalen)['c'].astype('float')[-1].tolist()
        currPrice2 = dataframe(leg2,timeframe,emalen)['c'].astype('float')[-1].tolist()
        
        currPL = amountLeg1*(currPrice1-priceLeg1) + amountLeg2*(currPrice2-priceLeg2)
        
        if exZscore <= 0:
            print ("Exiting! Current ZScore: {}". format(exZscore))
            order3 = exchange.createOrder (newLeg1, 'market', 'buy', amountLeg1)
            print("Long/ exited leg 1")
            print(order3)
            order4 = exchange.createOrder (newLeg2, 'market', 'sell', amountLeg2)
            print("Short/ exited leg 2")
            print(order4)
            print("-----")
            inPos = False
            print ("inPos status after exiting: {}". format(inPos))
        else:
            print ("Current ZScore: {}". format(exZscore))
            print("Sleeping 5 secs...")
            print("-----")
            time.sleep(5)
    
    "Calling entryConds for new buy session..."
    entryConds()

In [ ]:
# long buy
def longSide(leg1, leg2, legval):
    
    print("Long side function: ...")
    # amount for each leg
    priceLeg1 = dataframe(leg1,timeframe,emalen)['c'].astype('float')[-1].tolist()
    amountLeg1 = round(legval/priceLeg1,1)
    print ("Amount of leg1: {}". format(amountLeg1))
    
    priceLeg2 = dataframe(leg2,timeframe,emalen)['c'].astype('float')[-1].tolist()
    amountLeg2 = round(legval/priceLeg2,1)
    print ("Amount of leg2: {}". format(amountLeg2))
    print("-----")
        
    # long leg1
    order1 = exchange.createOrder(newLeg1, 'market', 'buy', amountLeg1)
    print("Longed leg 1")
    print(order1)
    # short leg 2
    order2 = exchange.createOrder(newLeg2, 'market', 'sell', amountLeg2)
    print("Shorted leg 2")
    print(order2)
    print("-----")
    
    inPos = True
    print ("inPos status: {}". format(inPos))
    
    while (inPos == True):
        exZscore = currZscore()
            
        if exZscore >= 0:
            print ("Exiting! Current ZScore: {}". format(exZscore))
            order3 = exchange.createOrder (newLeg1, 'market', 'sell', amountLeg1)
            print("Short/ exited leg 1")
            print(order3)
            order4 = exchange.createOrder (newLeg2, 'market', 'buy', amountLeg2)
            print("Long/ exited leg 2")
            print(order4)
            print("-----")
            inPos = False
            print ("inPos status after exiting: {}". format(inPos))
        else:
            print ("Current ZScore: {}". format(exZscore))
            print("Sleeping 5 secs...")
            print("-----")
            time.sleep(5)
    
    "Calling entryConds"
    entryConds()

In [ ]:
# entry conds
def entryConds():
    entered = False
    print ("Entry status at start of loop: {}". format(entered))
    print("-----")
    
    while entered == False:
        enZscore = currZscore()
        print ("Entry Zscore: {}". format(enZscore))

        if enZscore >= 2:
            print("Entering short side")
            print("-----")
            entered = True
            shortSide(leg1,leg2,legval)
        elif enZscore <= -2:
            print("Entering long side")
            print("-----")
            entered = True
            longSide(leg1,leg2,legval)
        else:
            print("No entry. Sleeping 5 secs...")
            print("-----")
            time.sleep(5)

entryConds()